In [21]:
import json
import re

import requests
import scrapy

ModuleNotFoundError: No module named 'scrapy'

In [ ]:
headers = {'User-Agent': 'UNC Journo Class'}


In [ ]:
base_url = 'http://goheels.com'
url = base_url + '/roster.aspx?path=baseball'

In [ ]:
resp = requests.get(url, headers=headers)

In [ ]:
body_str = resp.content.decode('utf-8')

In [ ]:
data = json.load(open('players-scraped.json'))

In [ ]:
sel = scrapy.Selector(text=body_str)

In [ ]:
table = sel.css('table')[0]

In [ ]:
cols = table.css('th').xpath('string()').extract()

In [ ]:
rows = table.css('tr')[1:]


In [ ]:
data_for_df = []

In [ ]:
players = []
for r in rows:
    data = {}
    for i, d in enumerate(r.css('td')):
        a = d.css('a')
        if a:
            t = a.xpath('text()').extract()[0]
            data['href'] = a.xpath('@href').extract()[0]
        else:
            t = d.xpath('text()').extract()[0]
        data[cols[i]] = t
    players.append(data)

In [ ]:
def fetch_bio(player):
    player_url = base_url + player['href']
    print('Fetch bio', player_url)
    resp = requests.get(player_url, headers=headers)
    player_txt = resp.content.decode('utf-8')
    sel = scrapy.Selector(text=player_txt)
    player['sel'] = sel
    player['bio'] = sel.css('#sidearm-roster-player-bio').xpath('string()').extract()[0]
    player['img'] = sel.css('.sidearm-roster-player-image img').xpath('@src').extract()[0]

In [ ]:
js_obj_rx = re.compile(r'.*?responsive-roster-bio\.ashx.*?(?P<obj>{.*?})')

In [ ]:
def fetch_stats(player):
    text = player['sel'].xpath('string()').extract()[0]
    parts = text.split('$.getJSON("/services/')[1:]
    captured = js_obj_rx.findall(''.join(parts))
    clean_objs = []
    for obj_str in captured:
        # We only want the stats object...
        if 'stats' not in obj_str:
            continue

        obj_str = obj_str.replace('{', '').replace('}', '')
        obj_str = obj_str.replace("'", '').replace('"', '')
        obj_pairs = obj_str.split(',')
        obj_pairs = [x.split(":") for x in obj_pairs]
        clean_pairs = []
        for pair in obj_pairs:
            clean_pairs.append(['"{}"'.format(p.strip()) for p in pair])
        colonized = [":".join(p) for p in clean_pairs]
        commas = ','.join(colonized)
        json_str = "{" + commas + "}"
        clean_objs.append(json.loads(json_str))
    
    player['stats_url'] = stats_url = (
        "http://goheels.com/services/responsive-roster-bio.ashx?"
        "type={type}&rp_id={rp_id}&path={path}&year={year}"
        "&player_id={player_id}"
    ).format(**clean_objs[0])
    
    print('Fetch stats', stats_url)
    resp = requests.get(stats_url, headers=headers)
    json_stats = json.loads(resp.content.decode("utf-8"))
    player['raw_stats'] = json_stats

In [ ]:

for p in players:
    fetch_bio(p)
    fetch_stats(p)

In [ ]:
players = []
for r in rows:
    data = {}
    for i, d in enumerate(r.css('td')):
        a = d.css('a')
        if a:
            t = a.xpath('text()').extract()[0]
            data['href'] = a.xpath('@href').extract()[0]
        else:
            t = d.xpath('text()').extract()[0]
        data[cols[i]] = t
    players.append(data)

In [ ]:
players[0]

In [ ]:
def fetch_bio(player):
    #unique player url
    player_url = base_url + player['href']
    #make request to player page
    resp = requests.get(player_url, headers=headers)
    #turn page into html
    player_txt = resp.content.decode('utf-8')
    #select the text on the page
    sel = scrapy.Selector(text=player_txt)
    player['sel'] = sel
    player['bio'] = sel.css('#sidearm-roster-player-bio').xpath('string()').extract()[0]
    player['img'] = sel.css('.sidearm-roster-player-image img').xpath('@src').extract()[0]

In [ ]:
p = [p for p in players if p['Full Name'] == 'Tyler Baum'][0]
txt = p['raw_stats']['career_stats']

In [ ]:
sel = scrapy.Selector(text=txt)
sel.css('section'

In [ ]:
def parse_stats(player):
    stats = {}
    for raw_key, raw_val in player['raw_stats'].items():
        txt = player['raw_stats'][raw_key]
        if not txt:
            print('Skipping {} for {}'.format(raw_key, player['Full Name']))
            continue
        sel = scrapy.Selector(text=txt)
        # Get all the tables
        for section in sel.css('section'):
            title = section.css('h5').xpath('string()').extract()[0]
            cols = section.css('tr')[0].css('th').xpath('string()').extract()
            print('NEW SECTION', title)
            print('COLS', cols)
            these_stats = []
            print('TRS', section.css('tr'))
            for r in section.css('tr')[1:]:
                print('row', r.xpath('string()').extract()[0].replace('\r', '').replace('\n', '').strip())
                s = {}
                for i, d in enumerate(r.css('td'), 1):
                    s[cols[i].lower()] = d.xpath('string()').extract()[0]
                yr = r.css('th').xpath('string()')
                if yr:
                    yr = yr.extract()[0]
                    if yr.lower() in ('total', 'season'):
                        print('SKIPPING...')
                        continue
                    print('THE YR IS', yr)
                    s['year'] = yr
                these_stats.append(s)
                print('THE STATS ARE', these_stats)
            existing = stats.get(raw_key, {})
            existing[title] = these_stats
            stats[raw_key] = existing
    player['stats'] = stats

In [ ]:
data[0]